In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
# get 24-25 Premier data

q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code,
        counties,
        CASE
            WHEN counties LIKE '%Broward%'
            OR counties LIKE '%Miami-Dade%'
            OR counties LIKE '%Palm Beach%'
            THEN 1
            ELSE 0
        END AS is_local
    FROM
        custom.cth_v_ticket_subscription_2425
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_subscription_2425.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Premier')
    GROUP BY
        purch_client_crm_id,
        zip,
        counties
),
touchpoints AS (
    SELECT DISTINCT
        a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-242024-25' THEN 4
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-242024-25' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2023-242024-25' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2024-25%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Premier')
        AND section_name != 'Panther Bar'
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2024-25'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2024-25'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal,
        COUNT(*) AS num_tickets
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Premier')
        AND season = '2024-25'
    GROUP BY
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal
),
location_ticket_type_agg AS (
    SELECT
        purchaser_ticketing_id,
        arena_level_internal
    FROM
        (SELECT
             purchaser_ticketing_id,
             arena_level_internal,
             ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
                 CASE arena_level_internal
                     WHEN 'Premium' THEN 1
                     WHEN 'Lower' THEN 2
                     WHEN 'Club' THEN 3
                     ELSE 4
                 END) AS rn
         FROM
             location_ticket_type)
    WHERE rn = 1
),
renewal_survey AS (
    SELECT
        formstack_v_stm_renewal_survey_2526.email,
        crm_id,
        how_likely_are_you_to_renew_your_territory_membership_for_next_season AS renewal_likelihood_survey
    FROM
        custom.formstack_v_stm_renewal_survey_2526
    LEFT JOIN
        custom.seatgeek_v_clients on formstack_v_stm_renewal_survey_2526.email = seatgeek_v_clients.email
)
SELECT
    zip.purch_client_crm_id,
    is_local,
    COALESCE(touchpoints_agg.touchpoints, 0) AS touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    CASE
        WHEN location_ticket_type_agg.arena_level_internal = 'Premium' THEN 1
        WHEN location_ticket_type_agg.arena_level_internal = 'Lowers' THEN 2
        WHEN location_ticket_type_agg.arena_level_internal = 'Clubs' THEN 3
        WHEN location_ticket_type_agg.arena_level_internal = 'Uppers' THEN 4
    END AS arena_level_internal,
    coalesce(renewal_likelihood_survey,'Did not Answer') as renewal_likelihood_survey
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type_agg ON zip.purch_client_crm_id = location_ticket_type_agg.purchaser_ticketing_id
LEFT JOIN
    renewal_survey ON zip.purch_client_crm_id = renewal_survey.crm_id
GROUP BY
    zip.purch_client_crm_id,
    zip.is_local,
    touchpoints,
    tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    gross_revenue,
    arena_level_internal,
    renewal_likelihood_survey
"""

df_premier_2425 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [4]:
# get 24-25 Full data

q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code,
        counties,
        CASE
            WHEN counties LIKE '%Broward%'
            OR counties LIKE '%Miami-Dade%'
            OR counties LIKE '%Palm Beach%'
            THEN 1
            ELSE 0
        END AS is_local
    FROM
        custom.cth_v_ticket_subscription_2425
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_subscription_2425.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Full')
    GROUP BY
        purch_client_crm_id,
        zip,
        counties
),
touchpoints AS (
    SELECT DISTINCT
        a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-242024-25' THEN 4
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-242024-25' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2023-242024-25' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2024-25%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Full')
        AND section_name != 'Panther Bar'
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2024-25'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2024-25'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal,
        COUNT(*) AS num_tickets
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Full')
        AND season = '2024-25'
    GROUP BY
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal
),
location_ticket_type_agg AS (
    SELECT
        purchaser_ticketing_id,
        arena_level_internal
    FROM
        (SELECT
             purchaser_ticketing_id,
             arena_level_internal,
             ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
                 CASE arena_level_internal
                     WHEN 'Premium' THEN 1
                     WHEN 'Lower' THEN 2
                     WHEN 'Club' THEN 3
                     ELSE 4
                 END) AS rn
         FROM
             location_ticket_type)
    WHERE rn = 1
),
renewal_survey AS (
    SELECT
        formstack_v_stm_renewal_survey_2526.email,
        crm_id,
        how_likely_are_you_to_renew_your_territory_membership_for_next_season AS renewal_likelihood_survey
    FROM
        custom.formstack_v_stm_renewal_survey_2526
    LEFT JOIN
        custom.seatgeek_v_clients on formstack_v_stm_renewal_survey_2526.email = seatgeek_v_clients.email
)
SELECT
    zip.purch_client_crm_id,
    zip.is_local,
    COALESCE(touchpoints_agg.touchpoints, 0) AS touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    CASE
        WHEN location_ticket_type_agg.arena_level_internal = 'Premium' THEN 1
        WHEN location_ticket_type_agg.arena_level_internal = 'Lowers' THEN 2
        WHEN location_ticket_type_agg.arena_level_internal = 'Clubs' THEN 3
        WHEN location_ticket_type_agg.arena_level_internal = 'Uppers' THEN 4
    END AS arena_level_internal,
    coalesce(renewal_likelihood_survey,'Did not Answer') as renewal_likelihood_survey
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type_agg ON zip.purch_client_crm_id = location_ticket_type_agg.purchaser_ticketing_id
LEFT JOIN
    renewal_survey ON zip.purch_client_crm_id = renewal_survey.crm_id
GROUP BY
    zip.purch_client_crm_id,
    zip.is_local,
    touchpoints,
    tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    gross_revenue,
    arena_level_internal,
    renewal_likelihood_survey
"""

df_full_2425 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [5]:
# get 23-24 Full data

q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code,
        counties,
        CASE
            WHEN counties LIKE '%Broward%'
            OR counties LIKE '%Miami-Dade%'
            OR counties LIKE '%Palm Beach%'
            THEN 1
            ELSE 0
        END AS is_local
    FROM
        custom.cth_v_ticket_subscription_2324
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_subscription_2324.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Full')
    GROUP BY
        purch_client_crm_id,
        section,
        zip,
        counties
),
touchpoints AS (
    SELECT DISTINCT
        a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2023'
        AND act.completed_on_coalesce < '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-24' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-24' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2023-24%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Full')
        and section_name != 'Panther Bar'
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2023-24'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2023-24'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal,
        count(*) as num_tickets
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Full')
        AND season = '2023-24'
    GROUP BY
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal
),
location_ticket_type_agg as (
    SELECT
        purchaser_ticketing_id,
        arena_level_internal
    FROM
        (SELECT
             purchaser_ticketing_id,
             arena_level_internal,
             ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
                 CASE arena_level_internal WHEN 'Premium' THEN 1 WHEN 'Lower' THEN 2 WHEN 'Club' THEN 3 ELSE 4 END) AS rn
         FROM
             location_ticket_type)
    WHERE rn = 1
),
did_renew AS (
    SELECT
        subs_2324.purch_client_crm_id,
        CASE
            WHEN subs_2425.purch_client_crm_id IS NULL THEN 0
            ELSE 1
        END AS did_renew
    FROM
        custom.cth_v_ticket_subscription_2324 subs_2324
    LEFT JOIN
        custom.cth_v_ticket_subscription_2425 subs_2425 ON subs_2324.purch_client_crm_id = subs_2425.purch_client_crm_id
    GROUP BY
        subs_2425.purch_client_crm_id,
        subs_2324.purch_client_crm_id
),
renewal_survey AS (
    SELECT
        formstack_v_stm_renewal_survey_2425.email,
        crm_id,
        how_likely_are_you_to_renew_your_territory_membership_for_next_season AS renewal_likelihood_survey
    FROM
        custom.formstack_v_stm_renewal_survey_2425
    LEFT JOIN
        custom.seatgeek_v_clients on formstack_v_stm_renewal_survey_2425.email = seatgeek_v_clients.email
)
SELECT
    zip.purch_client_crm_id,
    is_local,
    COALESCE(touchpoints_agg.touchpoints,0) AS touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    CASE
        WHEN location_ticket_type_agg.arena_level_internal = 'Premium' THEN 1
        WHEN location_ticket_type_agg.arena_level_internal = 'Lowers' THEN 2
        WHEN location_ticket_type_agg.arena_level_internal = 'Clubs' THEN 3
        WHEN location_ticket_type_agg.arena_level_internal = 'Uppers' THEN 4
    END AS arena_level_internal,
    did_renew.did_renew,
    coalesce(renewal_likelihood_survey,'Did not Answer') as renewal_likelihood_survey
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type_agg ON zip.purch_client_crm_id = location_ticket_type_agg.purchaser_ticketing_id
LEFT JOIN
    did_renew ON zip.purch_client_crm_id = did_renew.purch_client_crm_id
LEFT JOIN
    renewal_survey ON zip.purch_client_crm_id = renewal_survey.crm_id
GROUP BY
    zip.purch_client_crm_id,
    zip.is_local,
    touchpoints_agg.touchpoints,
    tenure,tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    gross_revenue,
    arena_level_internal,
    did_renew,
    renewal_likelihood_survey
"""

df_full_2324 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [6]:
# get 23-24 Premier data

Q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code,
        counties,
        CASE
            WHEN counties LIKE '%Broward%'
            OR counties LIKE '%Miami-Dade%'
            OR counties LIKE '%Palm Beach%'
            THEN 1
            ELSE 0
        END AS is_local
    FROM
        custom.cth_v_ticket_subscription_2324
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_subscription_2324.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Premier')
    GROUP BY
        purch_client_crm_id,
        section,
        zip,
        counties
),
touchpoints AS (
    SELECT
        DISTINCT a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2023'
        AND act.completed_on_coalesce < '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-24' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-24' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2023-24%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Premier')
        AND section_name != 'Panther Bar'
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2023-24'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2023-24'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal,
        count(*) as num_tickets
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Premier')
        AND season = '2023-24'
    GROUP BY
        purchaser_ticketing_id,
        ticket_type,
        arena_level_internal
),
location_ticket_type_agg as (
    SELECT
        purchaser_ticketing_id,
        arena_level_internal
    FROM
        (SELECT
             purchaser_ticketing_id,
             arena_level_internal,
             ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
                 CASE arena_level_internal WHEN 'Premium' THEN 1 WHEN 'Lower' THEN 2 WHEN 'Club' THEN 3 ELSE 4 END) AS rn
         FROM
             location_ticket_type)
    WHERE rn = 1
),
did_renew AS (
    SELECT
        subs_2324.purch_client_crm_id,
        CASE
            WHEN subs_2425.purch_client_crm_id IS NULL THEN 0
            ELSE 1
        END AS did_renew
    FROM
        custom.cth_v_ticket_subscription_2324 subs_2324
    LEFT JOIN
        custom.cth_v_ticket_subscription_2425 subs_2425 ON subs_2324.purch_client_crm_id = subs_2425.purch_client_crm_id
    GROUP BY
        subs_2425.purch_client_crm_id,
        subs_2324.purch_client_crm_id
),
renewal_survey AS (
    SELECT
        formstack_v_stm_renewal_survey_2425.email,
        crm_id,
        how_likely_are_you_to_renew_your_territory_membership_for_next_season AS renewal_likelihood_survey
    FROM
        custom.formstack_v_stm_renewal_survey_2425
    LEFT JOIN
        custom.seatgeek_v_clients on formstack_v_stm_renewal_survey_2425.email = seatgeek_v_clients.email
)
SELECT
    zip.purch_client_crm_id,
    is_local,
    COALESCE(touchpoints_agg.touchpoints,0) AS touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    CASE
        WHEN location_ticket_type_agg.arena_level_internal = 'Premium' THEN 1
        WHEN location_ticket_type_agg.arena_level_internal = 'Lowers' THEN 2
        WHEN location_ticket_type_agg.arena_level_internal = 'Clubs' THEN 3
        WHEN location_ticket_type_agg.arena_level_internal = 'Uppers' THEN 4
    END AS arena_level_internal,
    did_renew.did_renew,
    coalesce(renewal_likelihood_survey,'Did not Answer') as renewal_likelihood_survey
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type_agg ON zip.purch_client_crm_id = location_ticket_type_agg.purchaser_ticketing_id
LEFT JOIN
    did_renew ON zip.purch_client_crm_id = did_renew.purch_client_crm_id
LEFT JOIN
    renewal_survey ON zip.purch_client_crm_id = renewal_survey.crm_id
GROUP BY
    zip.purch_client_crm_id,
    zip.is_local,
    touchpoints_agg.touchpoints,
    tenure,tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    gross_revenue,
    arena_level_internal,
    did_renew,
    renewal_likelihood_survey
"""

df_premier_2324 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
# # drop any nulls

# df_premier_2324 = df_premier_2324.dropna(how = 'any',axis =0)
# df_full_2324 = df_full_2324.dropna(how = 'any',axis =0)

# df_premier_2425 = df_premier_2425.dropna(how = 'any',axis =0)
# df_full_2425 = df_full_2425.dropna(how = 'any',axis =0)

In [8]:
# make likelihood survey into numerical data 

survey_map = {
    'Extremely Likely': 3,
    'Likely': 2,
    'Neutral': 1,
    'Did not Answer': 0,
    'Unlikely': -1,
    'Extremely Unlikely': -2
}

df_premier_2324['renewal_likelihood_survey'] = df_premier_2324.apply(
    lambda row: survey_map.get(row['renewal_likelihood_survey'], 0), axis=1)

df_full_2324['renewal_likelihood_survey'] = df_full_2324.apply(
    lambda row: survey_map.get(row['renewal_likelihood_survey'], 0), axis=1)

df_premier_2425['renewal_likelihood_survey'] = df_premier_2425.apply(
    lambda row: survey_map.get(row['renewal_likelihood_survey'], 0), axis=1)

df_full_2425['renewal_likelihood_survey'] = df_full_2425.apply(
    lambda row: survey_map.get(row['renewal_likelihood_survey'], 0), axis=1)

In [9]:
# segment touchpoints 

df_premier_2324['touchpoints'] = pd.cut(df_premier_2324['touchpoints'], bins = [-np.inf,0,3,7,15,np.inf], labels = [0,1,2,3,4])

df_full_2324['touchpoints'] = pd.cut(df_full_2324['touchpoints'], bins = [-np.inf,0,3,7,15,np.inf], labels = [0,1,2,3,4])

df_premier_2425['touchpoints'] = pd.cut(df_premier_2425['touchpoints'], bins = [-np.inf,0,3,7,15,np.inf], labels = [0,1,2,3,4])

df_full_2425['touchpoints'] = pd.cut(df_full_2425['touchpoints'], bins = [-np.inf,0,3,7,15,np.inf], labels = [0,1,2,3,4])

In [10]:
# segment gross_revenue

df_premier_2324['gross_revenue'] = pd.cut(df_premier_2324['gross_revenue'], bins = [-np.inf,0,2500,5000,10000,20000,np.inf], labels = [0,1,2,3,4,5])

df_full_2324['gross_revenue'] = pd.cut(df_full_2324['gross_revenue'], bins = [-np.inf,0,5000,10000,20000,40000,np.inf], labels = [0,1,2,3,4,5])

df_premier_2425['gross_revenue'] = pd.cut(df_premier_2425['gross_revenue'], bins = [-np.inf,0,2500,5000,10000,20000,np.inf], labels = [0,1,2,3,4,5])

df_full_2425['gross_revenue'] = pd.cut(df_full_2425['gross_revenue'], bins = [-np.inf,0,5000,10000,20000,40000,np.inf], labels = [0,1,2,3,4,5])

In [11]:
# break into x and y data

x_premier_train = df_premier_2324[['is_local','touchpoints','tenure','real_show_rate','seat_show_rate','self_show_rate','num_games_attended','gross_revenue', 'arena_level_internal', 'renewal_likelihood_survey']]
y_premier = df_premier_2324[['did_renew']]

x_premier_test = df_premier_2425[['is_local','touchpoints','tenure','real_show_rate','seat_show_rate','self_show_rate','num_games_attended','gross_revenue', 'arena_level_internal', 'renewal_likelihood_survey']]
premier_acct_ids = df_premier_2425[['purch_client_crm_id']]

x_full_train = df_full_2324[['is_local', 'touchpoints','tenure','real_show_rate','seat_show_rate','self_show_rate','num_games_attended','gross_revenue', 'arena_level_internal', 'renewal_likelihood_survey']]
y_full = df_full_2324[['did_renew']]

x_full_test = df_full_2425[['is_local', 'touchpoints','tenure','real_show_rate','seat_show_rate','self_show_rate','num_games_attended','gross_revenue', 'arena_level_internal', 'renewal_likelihood_survey']]
full_acct_ids = df_full_2425[['purch_client_crm_id']]

In [12]:
# Premier Model

# ss = StandardScaler()
# x_train_scaler_premier = ss.fit_transform(x_premier_train)
# x_test_scaler_premier = ss.fit_transform(x_premier_test)

# sample_weights = np.ones(x_premier_train.shape[0])

# sample_weights[4] =  2
# sample_weights[9] =  2

# logi = LogisticRegression(random_state= 1993).fit(x_train_scaler_premier, np.array(y_premier), sample_weight=sample_weights)

In [13]:
# proba_predicted_premier = logi.predict_proba(x_test_scaler_premier)

# df_premier_2425['predicted_renew_percentage'] = list(map(lambda x: x[1]-.1, proba_predicted_premier))

In [14]:
# values, bins, bars = plt.hist(list(map(lambda x: x[1]-.1, proba_predicted_premier)), range = (0,1), bins = 10, rwidth = .9)
# plt.bar_label(bars)
# plt.show()

In [15]:
# Full Model

# ss = StandardScaler()
# x_train_scaler_full = ss.fit_transform(x_full_train)
# x_test_scaler_full = ss.fit_transform(x_full_test)

# logi = LogisticRegression().fit(x_train_scaler_full, np.array(y_full))

In [16]:
# proba_predicted_full = logi.predict_proba(x_test_scaler_full)

# #plt.hist(list(map(lambda x: x[1]-.1, proba_predictsd)))

# df_full_2425['predicted_renew_percentage'] = list(map(lambda x: x[1]-.1, proba_predicted_full))

In [17]:
# values, bins, bars = plt.hist(list(map(lambda x: x[1]-.1, proba_predicted_full)), range = (0,1), bins = 10, rwidth = .9)
# plt.bar_label(bars)
# plt.show()

In [18]:
# create monte carlo simulation for model

def run_log_reg(x_train, x_test, y, acct_ids, num_simulations):

    ss = StandardScaler()
    x_train_scaler = ss.fit_transform(x_train)
    x_test_scaler = ss.fit_transform(x_test)

    weights = [1,1,1,1.25,1,1,1,1,1,1.5]
    x_train_scaler = x_train_scaler*weights
    x_test_scaler = x_test_scaler*weights

    predicted_probs = np.zeros((x_test_scaler.shape[0], num_simulations))

    for i in range(num_simulations):

        logi = LogisticRegression().fit(x_train_scaler, np.array(y))
        predicted_probs[:,i] = logi.predict_proba(x_test_scaler)[:,1]
    
    avg_predicted_probs = np.mean(predicted_probs, axis = 1)

    result_df = x_test
    result_df['avg_predicted_prob'] = avg_predicted_probs
    result_df['acct_id'] = acct_ids

    return result_df

In [19]:
# run Full model

run_log_reg(x_full_train, x_full_test, y_full, full_acct_ids, 1000)

c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was 

,is_local,touchpoints,tenure,real_show_rate,seat_show_rate,self_show_rate,num_games_attended,gross_revenue,arena_level_internal,renewal_likelihood_survey,avg_predicted_prob,acct_id
0,0,0,4,0.000000,0.717742,0.000000,0,487.92,2,0,0.320881,2311
1,0,4,4,0.000000,0.971223,0.000000,0,39559.36,2,2,0.999993,124516
2,0,3,4,0.923077,0.923077,0.923077,23,2771.18,3,0,0.999420,510811
3,0,2,4,0.500000,0.916667,0.052083,4,4430.64,4,0,0.995814,546315
4,0,1,4,0.807692,0.900000,0.420000,8,7845.75,2,0,0.857332,547752
...,...,...,...,...,...,...,...,...,...,...,...,...
3494,0,0,1,0.600000,0.627451,0.549020,16,603.20,2,0,0.814970,25815752
3495,0,0,1,0.000000,0.942029,0.000000,0,4526.22,4,0,0.948288,25815811
3496,0,0,1,0.000000,0.927536,0.000000,0,4526.22,4,0,0.946014,25815864
3497,1,0,1,0.000000,0.947368,0.000000,0,6506.12,4,0,0.954857,25823117


In [20]:
# run Premier model

run_log_reg(x_premier_train, x_premier_test, y_premier, premier_acct_ids, 1000)

c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was 

,is_local,touchpoints,tenure,real_show_rate,seat_show_rate,self_show_rate,num_games_attended,gross_revenue,arena_level_internal,renewal_likelihood_survey,avg_predicted_prob,acct_id
0,0,4,3,0.590909,0.842105,0.171053,8,18456.98,2,0,0.999993,18064666
1,1,1,3,0.900000,0.923077,0.692308,9,4066.00,3,0,0.901259,16545038
2,0,2,1,0.625000,0.769231,0.307692,4,1819.00,4,0,0.999437,23244336
3,1,3,3,0.150000,0.291667,0.125000,3,7261.44,2,0,0.998727,19531127
4,1,2,2,0.200000,0.692308,0.000000,1,775.75,4,0,0.998680,17432238
...,...,...,...,...,...,...,...,...,...,...,...,...
1311,1,4,2,1.000000,1.000000,1.000000,14,3508.18,2,3,0.999998,24861174
1312,0,1,3,1.000000,0.916667,0.583333,7,1959.48,4,3,0.944809,17053490
1313,0,2,2,1.000000,1.000000,0.833333,10,3852.00,2,3,0.999102,11835255
1314,0,0,3,0.777778,0.846154,0.538462,7,963.00,4,0,0.311395,16546046
